In [1]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.2/400.2 kB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.1 MB/s eta 0:00:00
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [457 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Hit:8 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,235 kB]
Hit:10 https://ppa.launchpa

In [2]:
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common import NoSuchElementException


def navegador():
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('window-size=1000, 1000')
    chrome_options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=chrome_options)
    return driver


def mostrar_mais_jogos(driver):
    xpaths = [
        '//*[@id="detail"]/div[6]/div[2]/div[1]/div[3]',
        '//*[@id="detail"]/div[7]/div[2]/div[1]/div[3]',
        '//*[@id="detail"]/div[8]/div[2]/div[1]/div[3]',
        '//*[@id="detail"]/div[9]/div[2]/div[1]/div[3]'
    ]
    for xpath in xpaths:
        try:
            element = driver.find_element(By.XPATH, xpath)
            driver.execute_script("arguments[0].click();", element)
        except NoSuchElementException:
            pass


def pegar_resultados(driver, url):
    # Acessar a URL e esperar um pouco para garantir que a página seja carregada
    driver.get(url)
    sleep(1)

    # Encontrar os elementos de time mandante e visitante
    time_mandante_element = driver.find_element(By.XPATH, '//*[@id="detail"]/div[5]/div[4]/div[3]/div[1]/a')
    time_visitante_element = driver.find_element(By.XPATH, '//*[@id="detail"]/div[5]/div[2]/div[3]/div[2]/a')
    time_mandante = time_mandante_element.text.split('(')[0]
    time_visitante = time_visitante_element.text.split('(')[0]
    times = f'{time_visitante} vs {time_mandante}'

    # Mostrar mais jogos para carregar os resultados completos
    mostrar_mais_jogos(driver)

    # Obter elementos de resultados e gols das partidas
    elementos_resultados = driver.find_elements(By.CSS_SELECTOR, 'span.h2h__icon > div')
    elementos_gols = driver.find_elements(By.CSS_SELECTOR, 'span.h2h__result')
    elementos_gols_full = driver.find_elements(By.CSS_SELECTOR, 'span.h2h__result__fulltime')

    resultados_partidas = []
    gols_partidas = []

    # Processar os elementos de resultados e gols
    for elemento_resultado, elemento_gols, elemento_gols_full in zip(elementos_resultados, elementos_gols,
                                                                     elementos_gols_full):
        titulo = elemento_resultado.get_attribute('title')
        resultado = elemento_resultado.text
        gols = list(map(int, elemento_gols.text.split()))

        if '/' in titulo and titulo.split('/')[0] == 'Empate':
            gols_full = list(map(int, elemento_gols_full.text.split()))
            resultados_partidas.append('E')
            gols_partidas.append(gols_full)
        else:
            resultados_partidas.append(resultado)
            gols_partidas.append(gols)

    return resultados_partidas[: 10], gols_partidas[: 10], times


In [8]:
driver = navegador()
driver.get('https://www.livescore.in/br/')
sleep(2)

# Clique no botão 'Jogos de amanhã'
driver.find_element(By.XPATH, '//*[@id="live-table"]/div[1]/div[2]/div/button[3]').click()
sleep(1)

# Clica no menu e configura o fuso horário do Brasil
driver.find_element(By.CSS_SELECTOR, '.header__block--menu div.header__button').click()
driver.find_element(By.CSS_SELECTOR, 'div.contextMenu__row').click()
driver.find_element(By.CSS_SELECTOR, 'div.timeZone__button').click()
driver.find_element(By.XPATH, '//*[@id="tzContentList"]/div[9]').click()

In [10]:
# Obtém os IDs das partidas
ids = []

elementos = driver.find_elements(By.CSS_SELECTOR, 'div.event__match')
for elemento in elementos:
    id_partida = elemento.get_attribute('id').split('_')[2]
    ids.append(id_partida)

In [12]:
for id in ids:
    try:
        url_mandante = f'https://www.livescore.in/br/jogo/{id}/#/h2h/home'
        resultados_partidas_m, gols_partidas_m, times = pegar_resultados(driver, url_mandante)

        # Verifica se há 10 resultados para o time mandante
        if len(resultados_partidas_m) == 10:
            url_visitante = f'https://www.livescore.in/br/jogo/{id}/#/h2h/away'
            resultados_partidas_v, gols_partidas_v, times = pegar_resultados(driver, url_visitante)

            # Verifica se há 10 resultados para o time visitante
            if len(resultados_partidas_v) == 10:
                print(f'{times}\n{gols_partidas_m}\n{resultados_partidas_m}\n' \
                      f'{gols_partidas_v}\n{resultados_partidas_v}\n')
            else:
                pass
        else:
            pass
    except ValueError:  # Erro causado por '-', ou seja, em casos em que a partida não tem gols.
        pass

driver.quit()

Novorizontino vs Mirassol
[[3, 1], [3, 0], [2, 2], [0, 1], [1, 0], [1, 0], [2, 0], [0, 1], [0, 0], [1, 0]]
['V', 'V', 'E', 'D', 'V', 'V', 'V', 'D', 'E', 'V']
[[0, 2], [0, 1], [2, 2], [0, 3], [0, 0], [1, 1], [2, 1], [1, 2], [2, 0], [1, 0]]
['V', 'V', 'E', 'V', 'E', 'E', 'D', 'V', 'D', 'D']

Tombense vs Sport
[[2, 2], [2, 1], [0, 1], [4, 2], [1, 2], [1, 2], [1, 1], [0, 2], [0, 1], [1, 1]]
['E', 'V', 'D', 'V', 'D', 'D', 'E', 'D', 'D', 'E']
[[0, 1], [1, 2], [3, 1], [2, 0], [1, 1], [1, 1], [1, 2], [2, 3], [1, 0], [1, 1]]
['V', 'V', 'D', 'D', 'E', 'E', 'V', 'V', 'D', 'E']

Almería vs Rayo Vallecano
[[1, 0], [1, 1], [0, 1], [0, 0], [3, 0], [2, 1], [1, 2], [2, 1], [1, 1], [0, 2]]
['V', 'E', 'D', 'E', 'V', 'V', 'D', 'V', 'E', 'D']
[[1, 1], [3, 2], [1, 1], [1, 1], [0, 1], [3, 0], [2, 1], [3, 1], [4, 0], [2, 1]]
['E', 'D', 'E', 'E', 'V', 'D', 'D', 'D', 'D', 'D']

Sevilla vs Valencia
[[1, 0], [1, 1], [5, 2], [1, 1], [1, 2], [0, 0], [1, 1], [3, 2], [0, 2], [2, 1]]
['V', 'E', 'V', 'E', 'D', 'E', 'E'